## Import Required Packages


In [ ]:
# Import Python packages
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Import Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

# Import Snowpark ML
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LinearRegression, LogisticRegression
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_error, accuracy_score, roc_auc_score
from snowflake.ml.registry import Registry

print("✅ Packages imported successfully")


## Connect to Snowflake

Get active session and set context to Snow Health database.


In [ ]:
# Get active Snowflake session
session = get_active_session()

# Set context
session.use_database('SNOW_HEALTH_INTELLIGENCE')
session.use_schema('ANALYTICS')
session.use_warehouse('SNOW_HEALTH_WH')

print(f"✅ Connected - Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Database.Schema: {session.get_fully_qualified_current_schema()}")


---
# MODEL 1: Compliance Risk Prediction

Predict which employees are at high risk of compliance violations based on training completion, credential status, and policy acknowledgment patterns.


### Prepare Compliance Risk Training Data


In [ ]:
# Get employee compliance features
compliance_df = session.sql("""
SELECT
    e.employee_id,
    o.organization_type,
    e.job_role,
    e.department,
    -- Training compliance metrics
    COUNT(DISTINCT ce.enrollment_id)::FLOAT AS total_enrollments,
    COUNT(DISTINCT cc.completion_id)::FLOAT AS total_completions,
    AVG(CASE WHEN cc.completion_status = 'COMPLETED' THEN cc.score ELSE 0 END)::FLOAT AS avg_score,
    COUNT(DISTINCT CASE WHEN ce.due_date < CURRENT_DATE() 
                       AND cc.completion_id IS NULL 
                       THEN ce.enrollment_id END)::FLOAT AS overdue_courses,
    -- Credential compliance
    COUNT(DISTINCT cr.credential_id)::FLOAT AS credential_count,
    COUNT(DISTINCT CASE WHEN cr.expiration_date < DATEADD('day', 90, CURRENT_DATE())
                       THEN cr.credential_id END)::FLOAT AS expiring_credentials,
    -- Policy acknowledgment
    COUNT(DISTINCT pa.acknowledgment_id)::FLOAT AS policies_acknowledged,
    AVG(DATEDIFF('day', pa.acknowledgment_date, CURRENT_DATE()))::FLOAT AS avg_days_since_ack,
    -- Incident history
    COUNT(DISTINCT i.incident_id)::FLOAT AS incident_count,
    -- Target: Has compliance issues
    (COUNT(DISTINCT CASE WHEN ce.due_date < CURRENT_DATE() AND cc.completion_id IS NULL THEN ce.enrollment_id END) > 0
     OR COUNT(DISTINCT CASE WHEN cr.expiration_date < DATEADD('day', 30, CURRENT_DATE()) THEN cr.credential_id END) > 0
     OR COUNT(DISTINCT CASE WHEN i.incident_date >= DATEADD('month', -6, CURRENT_DATE()) THEN i.incident_id END) > 0)::BOOLEAN AS has_compliance_risk
FROM RAW.EMPLOYEES e
JOIN RAW.ORGANIZATIONS o ON e.organization_id = o.organization_id
LEFT JOIN RAW.COURSE_ENROLLMENTS ce ON e.employee_id = ce.employee_id
LEFT JOIN RAW.COURSE_COMPLETIONS cc ON ce.enrollment_id = cc.enrollment_id
LEFT JOIN RAW.CREDENTIALS cr ON e.employee_id = cr.employee_id
LEFT JOIN RAW.POLICY_ACKNOWLEDGMENTS pa ON e.employee_id = pa.employee_id
LEFT JOIN RAW.INCIDENTS i ON e.employee_id = i.reported_by_employee_id
WHERE e.employee_status = 'ACTIVE'
GROUP BY e.employee_id, o.organization_type, e.job_role, e.department
HAVING COUNT(DISTINCT ce.enrollment_id) > 2
LIMIT 50000
""")

print(f"Compliance data: {compliance_df.count()} employees")
compliance_df.show(5)


### Split Data and Train Compliance Risk Model


In [ ]:
# Train/test split (80/20)
train_compliance, test_compliance = compliance_df.random_split([0.8, 0.2], seed=42)

# Drop EMPLOYEE_ID (VARCHAR not supported as feature)
train_compliance = train_compliance.drop("EMPLOYEE_ID")
test_compliance = test_compliance.drop("EMPLOYEE_ID")

# Create pipeline
compliance_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["ORGANIZATION_TYPE", "JOB_ROLE", "DEPARTMENT"],
        output_cols=["ORG_TYPE_ENC", "JOB_ROLE_ENC", "DEPARTMENT_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["HAS_COMPLIANCE_RISK"],
        output_cols=["COMPLIANCE_RISK_PREDICTION"],
        n_estimators=100,
        max_depth=10
    ))
])

# Train model
compliance_pipeline.fit(train_compliance)
print("✅ Compliance risk prediction model trained")


### Evaluate and Register Compliance Risk Model


In [ ]:
# Make predictions on test set
test_predictions = compliance_pipeline.predict(test_compliance)

# Calculate metrics
accuracy = accuracy_score(df=test_predictions, y_true_col_names="HAS_COMPLIANCE_RISK", y_pred_col_names="COMPLIANCE_RISK_PREDICTION")
metrics = {"accuracy": round(accuracy, 4)}
print(f"Model metrics: {metrics}")

# Register model
reg = Registry(session)
reg.log_model(
    model=compliance_pipeline,
    model_name="COMPLIANCE_RISK_PREDICTOR",
    version_name="V1",
    comment="Predicts employee compliance risk based on training, credentials, and policy acknowledgment patterns using Random Forest",
    metrics=metrics
)

print("✅ Compliance risk model registered to Model Registry as COMPLIANCE_RISK_PREDICTOR")


---
# MODEL 2: Course Completion Prediction

Predict which enrolled employees are likely to complete their training courses on time.


### Prepare Course Completion Training Data


In [ ]:
# Get enrollment features for completion prediction
completion_df = session.sql("""
SELECT
    ce.enrollment_id,
    c.course_category,
    c.course_type,
    e.job_role,
    e.department,
    o.organization_type,
    DATEDIFF('day', ce.enrollment_date, ce.due_date)::FLOAT AS days_to_complete,
    DATEDIFF('day', ce.enrollment_date, CURRENT_DATE())::FLOAT AS days_since_enrollment,
    -- Historical completion rate for this employee
    (SELECT COUNT(DISTINCT cc2.completion_id)::FLOAT / NULLIF(COUNT(DISTINCT ce2.enrollment_id)::FLOAT, 0)
     FROM RAW.COURSE_ENROLLMENTS ce2
     LEFT JOIN RAW.COURSE_COMPLETIONS cc2 ON ce2.enrollment_id = cc2.enrollment_id
     WHERE ce2.employee_id = e.employee_id
       AND ce2.enrollment_date < ce.enrollment_date) AS historical_completion_rate,
    -- Target: Was completed
    (cc.completion_status = 'COMPLETED')::BOOLEAN AS was_completed
FROM RAW.COURSE_ENROLLMENTS ce
JOIN RAW.COURSES c ON ce.course_id = c.course_id
JOIN RAW.EMPLOYEES e ON ce.employee_id = e.employee_id
JOIN RAW.ORGANIZATIONS o ON e.organization_id = o.organization_id
LEFT JOIN RAW.COURSE_COMPLETIONS cc ON ce.enrollment_id = cc.enrollment_id
WHERE ce.enrollment_date >= DATEADD('month', -12, CURRENT_DATE())
  AND ce.due_date < CURRENT_DATE()
LIMIT 100000
""")

print(f"Course completion data: {completion_df.count()} enrollments")
completion_df.show(5)


### Train Course Completion Model


In [ ]:
# Split data
train_completion, test_completion = completion_df.random_split([0.8, 0.2], seed=42)

# Drop ENROLLMENT_ID
train_completion = train_completion.drop("ENROLLMENT_ID")
test_completion = test_completion.drop("ENROLLMENT_ID")

# Create pipeline
completion_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["COURSE_CATEGORY", "COURSE_TYPE", "JOB_ROLE", "DEPARTMENT", "ORGANIZATION_TYPE"],
        output_cols=["COURSE_CAT_ENC", "COURSE_TYPE_ENC", "JOB_ROLE_ENC", "DEPT_ENC", "ORG_TYPE_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", LogisticRegression(
        label_cols=["WAS_COMPLETED"],
        output_cols=["COMPLETION_PREDICTION"]
    ))
])

# Train
completion_pipeline.fit(train_completion)
print("✅ Course completion prediction model trained")


### Evaluate and Register Completion Model


In [ ]:
# Predict on test set
completion_predictions = completion_pipeline.predict(test_completion)

# Calculate accuracy
comp_accuracy = accuracy_score(df=completion_predictions, 
                                y_true_col_names="WAS_COMPLETED",
                                y_pred_col_names="COMPLETION_PREDICTION")
comp_metrics = {"accuracy": round(comp_accuracy, 4)}
print(f"Completion model metrics: {comp_metrics}")

# Register model
reg.log_model(
    model=completion_pipeline,
    model_name="COURSE_COMPLETION_PREDICTOR",
    version_name="V1",
    comment="Predicts course completion likelihood using Logistic Regression based on employee and course characteristics",
    metrics=comp_metrics
)

print("✅ Course completion model registered to Model Registry as COURSE_COMPLETION_PREDICTOR")


---
# MODEL 3: Subscription Churn Prediction

Predict which healthcare organizations are likely to cancel their Snow Health subscriptions.


### Prepare Subscription Churn Data


In [ ]:
# Get subscription features for churn prediction
churn_df = session.sql("""
SELECT
    o.organization_id,
    o.organization_type,
    s.service_type,
    s.subscription_tier,
    o.lifetime_value::FLOAT AS lifetime_value,
    o.compliance_risk_score::FLOAT AS compliance_risk_score,
    DATEDIFF('month', s.start_date, CURRENT_DATE())::FLOAT AS subscription_age_months,
    s.monthly_price::FLOAT AS monthly_price,
    -- Usage metrics
    COUNT(DISTINCT e.employee_id)::FLOAT AS active_employees,
    COUNT(DISTINCT ce.enrollment_id)::FLOAT AS total_enrollments,
    COUNT(DISTINCT st.ticket_id)::FLOAT AS support_tickets,
    AVG(st.customer_satisfaction_score)::FLOAT AS avg_csat,
    -- Transaction activity
    COUNT(DISTINCT CASE WHEN t.transaction_date >= DATEADD('month', -3, CURRENT_DATE())
                       THEN t.transaction_id END)::FLOAT AS recent_transactions,
    -- Incidents
    COUNT(DISTINCT i.incident_id)::FLOAT AS incident_count,
    -- Target: Churned
    (s.subscription_status = 'CANCELLED' 
     OR (s.end_date IS NOT NULL AND s.end_date < CURRENT_DATE()))::BOOLEAN AS is_churned
FROM RAW.ORGANIZATIONS o
JOIN RAW.SUBSCRIPTIONS s ON o.organization_id = s.organization_id
LEFT JOIN RAW.EMPLOYEES e ON o.organization_id = e.organization_id AND e.employee_status = 'ACTIVE'
LEFT JOIN RAW.COURSE_ENROLLMENTS ce ON e.employee_id = ce.employee_id
LEFT JOIN RAW.SUPPORT_TICKETS st ON o.organization_id = st.organization_id
LEFT JOIN RAW.TRANSACTIONS t ON s.subscription_id = t.subscription_id
LEFT JOIN RAW.INCIDENTS i ON o.organization_id = i.organization_id
WHERE s.start_date >= DATEADD('month', -24, CURRENT_DATE())
GROUP BY o.organization_id, o.organization_type, s.service_type, s.subscription_tier, 
         o.lifetime_value, o.compliance_risk_score, s.start_date, s.monthly_price, 
         s.subscription_status, s.end_date
HAVING COUNT(DISTINCT e.employee_id) > 5
LIMIT 25000
""")

print(f"Subscription churn data: {churn_df.count()} subscriptions")
churn_df.show(5)


### Train Subscription Churn Model


In [ ]:
# Split data
train_churn, test_churn = churn_df.random_split([0.8, 0.2], seed=42)

# Drop ORGANIZATION_ID
train_churn = train_churn.drop("ORGANIZATION_ID")
test_churn = test_churn.drop("ORGANIZATION_ID")

# Create pipeline
churn_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["ORGANIZATION_TYPE", "SERVICE_TYPE", "SUBSCRIPTION_TIER"],
        output_cols=["ORG_TYPE_ENC", "SERVICE_TYPE_ENC", "SUB_TIER_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["IS_CHURNED"],
        output_cols=["CHURN_PREDICTION"],
        n_estimators=100,
        max_depth=12
    ))
])

# Train
churn_pipeline.fit(train_churn)
print("✅ Subscription churn prediction model trained")


### Evaluate and Register Churn Model


In [ ]:
# Predict on test set
churn_predictions = churn_pipeline.predict(test_churn)

# Calculate accuracy
churn_accuracy = accuracy_score(df=churn_predictions, 
                                 y_true_col_names="IS_CHURNED",
                                 y_pred_col_names="CHURN_PREDICTION")
churn_metrics = {"accuracy": round(churn_accuracy, 4)}
print(f"Churn model metrics: {churn_metrics}")

# Register model
reg.log_model(
    model=churn_pipeline,
    model_name="SUBSCRIPTION_CHURN_PREDICTOR",
    version_name="V1",
    comment="Predicts subscription churn probability using Random Forest based on usage patterns and engagement",
    metrics=churn_metrics
)

print("✅ Subscription churn model registered to Model Registry as SUBSCRIPTION_CHURN_PREDICTOR")


---
# Verify Models in Registry


In [ ]:
# Show all models in the registry
print("Models in registry:")
reg.show_models()

# Show versions for compliance risk model
print("\nCompliance risk model versions:")
reg.get_model("COMPLIANCE_RISK_PREDICTOR").show_versions()

# Show versions for course completion model  
print("\nCourse completion model versions:")
reg.get_model("COURSE_COMPLETION_PREDICTOR").show_versions()

# Show versions for churn model
print("\nSubscription churn model versions:")
reg.get_model("SUBSCRIPTION_CHURN_PREDICTOR").show_versions()

print("\n✅ All models registered and ready to add to Intelligence Agent")


---
# Test Model Inference

Test calling each model to make predictions.


In [ ]:
# Test compliance risk prediction on sample employees
compliance_model = reg.get_model("COMPLIANCE_RISK_PREDICTOR").default
sample_employees = compliance_df.limit(5).drop("EMPLOYEE_ID")
compliance_preds = compliance_model.run(sample_employees, function_name="predict")
print("Compliance risk predictions:")
compliance_preds.select("HAS_COMPLIANCE_RISK", "COMPLIANCE_RISK_PREDICTION").show()

# Test course completion prediction
completion_model = reg.get_model("COURSE_COMPLETION_PREDICTOR").default
sample_enrollments = completion_df.limit(5).drop("ENROLLMENT_ID")
completion_preds = completion_model.run(sample_enrollments, function_name="predict")
print("\nCourse completion predictions:")
completion_preds.select("WAS_COMPLETED", "COMPLETION_PREDICTION").show()

# Test subscription churn prediction
churn_model = reg.get_model("SUBSCRIPTION_CHURN_PREDICTOR").default
sample_subscriptions = churn_df.limit(5).drop("ORGANIZATION_ID")
churn_preds = churn_model.run(sample_subscriptions, function_name="predict")
print("\nSubscription churn predictions:")
churn_preds.select("IS_CHURNED", "CHURN_PREDICTION").show()

print("\n✅ All models tested successfully!")
